# 1. Import All Required Libraries
*This section loads all essential libraries for data processing, text cleaning, machine learning, visualization, and model evaluation.*


In [ ]:
import pandas as pd
import numpy as np
import re
import os
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

from collections import Counter
import joblib

print("✅ All libraries loaded successfully")


✅ All libraries loaded successfully


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 2. Load the Jigsaw Toxic Comment Dataset
*We load a real-world dataset that contains labeled toxic and non-toxic comments. This dataset is used to train our machine learning model.*


In [ ]:
dataset_path = "/content/train.csv"   # change to "train.csv" if running locally

raw_dataset = pd.read_csv(dataset_path, engine='python', on_bad_lines='skip')

toxic_label_columns = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

raw_dataset["toxicity_label"] = (raw_dataset[toxic_label_columns].sum(axis=1) > 0).astype(int)

dataset = raw_dataset[["comment_text", "toxicity_label"]]
dataset.columns = ["comment_text", "toxicity_label"]

print(dataset.head())
print("\nLabel Distribution:")
print(dataset["toxicity_label"].value_counts())


                                        comment_text  toxicity_label
0  Explanation\nWhy the edits made under my usern...               0
1  D'aww! He matches this background colour I'm s...               0
2  Hey man, I'm really not trying to edit war. It...               0
3  "\nMore\nI can't make any real suggestions on ...               0
4  You, sir, are my hero. Any chance you remember...               0

Label Distribution:
toxicity_label
0    143346
1     16225
Name: count, dtype: int64


# 3. Balance the Dataset
*The dataset contains more non-toxic comments than toxic ones. We balance the data so the model learns both classes equally and avoids bias.*


In [ ]:
toxic_comments = dataset[dataset["toxicity_label"] == 1]
non_toxic_comments = dataset[dataset["toxicity_label"] == 0]

toxic_count = len(toxic_comments)

balanced_dataset = pd.concat([
    toxic_comments,
    non_toxic_comments.sample(toxic_count, random_state=42)
])

balanced_dataset = balanced_dataset.sample(frac=1, random_state=42).reset_index(drop=True)

print("✅ Balanced Dataset Distribution:")
print(balanced_dataset["toxicity_label"].value_counts())


✅ Balanced Dataset Distribution:
toxicity_label
1    16225
0    16225
Name: count, dtype: int64


# 4. Text Preprocessing Function
*This function removes URLs, special characters, emojis, and converts the text into a clean format suitable for machine learning.*


In [ ]:
def preprocess_comment(comment_text):
    comment_text = comment_text.lower()
    comment_text = re.sub(r"http\S+|@\w+|#\w+", "", comment_text)
    comment_text = re.sub(r"[^\u0C00-\u0C7Fa-zA-Z\s]", "", comment_text)
    comment_text = re.sub(r"\s+", " ", comment_text).strip()
    return comment_text


# 5. Normalize Common Misspellings
*Users intentionally misspell abusive words (e.g., idoit → idiot). This step improves detection accuracy.*


In [ ]:
COMMON_MISSPELLING_MAP = {
    "idoit": "idiot",
    "stupit": "stupid",
    "fuk": "fuck",
    "fck": "fuck",
    "bich": "bitch",
    "asshloe": "asshole"
}

def normalize_slang_words(clean_text):
    words = clean_text.split()
    corrected_words = [COMMON_MISSPELLING_MAP.get(word, word) for word in words]
    return " ".join(corrected_words)


# 6. Apply Preprocessing to the Entire Dataset
*We clean and normalize every comment before training the model.*


In [ ]:
balanced_dataset["cleaned_comment"] = balanced_dataset["comment_text"].apply(preprocess_comment)
balanced_dataset["cleaned_comment"] = balanced_dataset["cleaned_comment"].apply(normalize_slang_words)

balanced_dataset.head()


,comment_text,toxicity_label,cleaned_comment
0,You are an old cougar! You are an old cougar!,1,you are an old cougar you are an old cougar
1,I'm scared\nbrrrr.... i'm gonna die now,0,im scared brrrr im gonna die now
2,"Okay, so anal sex is as widely unaccepted and ...",1,okay so anal sex is as widely unaccepted and d...
3,shut up you cunt\nWWWWWWWWWWWWWWWWWWWWWWWWWWWW...,1,shut up you cunt wwwwwwwwwwwwwwwwwwwwwwwwwwwww...
4,You arrogant administrator homosexual bastards...,1,you arrogant administrator homosexual bastards...


# 7. Split Dataset into Train & Test Sets
*We divide our dataset into training and testing sets to evaluate how well the model performs on unseen data.*


In [ ]:
feature_text = balanced_dataset["cleaned_comment"]
target_labels = balanced_dataset["toxicity_label"]

training_text, testing_text, training_labels, testing_labels = train_test_split(
    feature_text, target_labels, test_size=0.2, random_state=42, stratify=target_labels
)


# 8. Train Machine Learning Model (TF-IDF + Logistic Regression)
*TF-IDF converts text into numerical features and Logistic Regression is used to classify comments as toxic or non-toxic.*


In [ ]:
toxicity_detection_pipeline = Pipeline([
    ("tfidf_vectorizer", TfidfVectorizer(
        ngram_range=(1, 3),
        max_features=15000,
        min_df=3,
        max_df=0.9,
        sublinear_tf=True
    )),
    ("logistic_classifier", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        solver="lbfgs"
    ))
])

toxicity_detection_pipeline.fit(training_text, training_labels)

predicted_labels = toxicity_detection_pipeline.predict(testing_text)

print(classification_report(testing_labels, predicted_labels))


              precision    recall  f1-score   support

           0       0.87      0.91      0.89      3245
           1       0.91      0.86      0.88      3245

    accuracy                           0.89      6490
   macro avg       0.89      0.89      0.89      6490
weighted avg       0.89      0.89      0.89      6490



# 9. Initial Toxicity Test Before Masking
*We test the trained model using a custom comment to verify that toxicity detection is working correctly.*


In [ ]:
input_comment = "fuck you idoit"

cleaned_input_comment = normalize_slang_words(preprocess_comment(input_comment))

prediction_probability = toxicity_detection_pipeline.predict_proba([cleaned_input_comment])[0][1]
prediction_label = toxicity_detection_pipeline.predict([cleaned_input_comment])[0]

print("Prediction:", "Toxic" if prediction_label == 1 else "Non-Toxic")
print("Score:", round(prediction_probability * 100, 2), "%")


Prediction: Toxic
Score: 99.99 %


# 10. Auto-Extract Offensive Words From Toxic Comments
*We automatically extract the most commonly used abusive words from real toxic comments in the dataset.*


In [ ]:
def extract_top_toxic_words_from_dataset(balanced_dataset, top_n=300):
    toxic_sentences = balanced_dataset[balanced_dataset["toxicity_label"] == 1]["cleaned_comment"]
    combined_words = " ".join(toxic_sentences).split()
    filtered_words = [word for word in combined_words if len(word) > 2]
    word_counter = Counter(filtered_words)
    return [word for word, count in word_counter.most_common(top_n)]

auto_extracted_bad_words = extract_top_toxic_words_from_dataset(balanced_dataset)


# 11. Add Extra English & Telugu Offensive Words
*To ensure complete coverage, we manually add commonly known abusive words that may not appear frequently in the dataset.*


In [ ]:
english_offensive_words = [
    "slut","whore","dick","cock","pussy","cunt","hoe",
    "retard","douche","prick","motherfucker","mf",
    "asswipe","jerk","fucktard"
]

telugu_offensive_words = [
    "దొంగ", "మూర్ఖుడు", "నీచుడు", "లంజ", "దెంగు",
    "పిచ్చోడు", "అసహ్యం", "చెత్తోడు", "బుద్ధిలేని",
    "దుర్మార్గుడు","వెధవ"
]


# 12. Generate Disguised / Leet Variants of Offensive Words
*Users hide abusive words using symbols and numbers (f*ck, idi0t). This function detects those variations.*


In [ ]:
def generate_disguised_variants(base_word):
    variants = set()
    variants.add(base_word)

    leet_replacements = {
        "a": ["@", "4"],
        "i": ["1", "!"],
        "e": ["3"],
        "o": ["0"],
        "s": ["$", "5"],
        "t": ["7"]
    }

    for index, character in enumerate(base_word):
        if character in leet_replacements:
            for replacement in leet_replacements[character]:
                variants.add(base_word[:index] + replacement + base_word[index+1:])

    variants.add(base_word + base_word[-1])
    variants.add(".".join(base_word))

    if len(base_word) > 3:
        variants.add(base_word[0] + "*"*(len(base_word)-2) + base_word[-1])

    return variants


# 13. Build the Final Industry-Grade Offensive Word Set
*We combine dataset-extracted words, English and Telugu lists, and disguised variants to form a powerful masking dictionary.*


In [ ]:
final_offensive_word_list = set()

for word in auto_extracted_bad_words:
    final_offensive_word_list.add(word)
    final_offensive_word_list.update(generate_disguised_variants(word))

for word in english_offensive_words:
    final_offensive_word_list.add(word)
    final_offensive_word_list.update(generate_disguised_variants(word))

for word in telugu_offensive_words:
    final_offensive_word_list.add(word)

final_offensive_word_list = set([word.lower() for word in final_offensive_word_list if isinstance(word, str)])

print("✅ Total Offensive Patterns:", len(final_offensive_word_list))


✅ Total Offensive Patterns: 2159


# 14. Precompile Regex Patterns for Fast Masking
*Precompiling regex patterns improves the performance of real-time masking.*


In [ ]:
compiled_masking_patterns = []

for word in final_offensive_word_list:
    if len(word) <= 2:
        continue
    compiled_masking_patterns.append(re.compile(re.escape(word), flags=re.IGNORECASE))


# 15. Final Advanced Masking Function
*This function replaces every detected abusive word with asterisks of equal length.*


In [ ]:
def advanced_toxic_word_masking(original_text):
    masked_text = original_text
    for pattern in compiled_masking_patterns:
        masked_text = pattern.sub(lambda match: "*" * len(match.group(0)), masked_text)
    return masked_text


# 16. Final Detection + Masking Pipeline
*This function combines machine learning toxicity detection and rule-based masking into one complete pipeline.*


In [ ]:
def detect_and_mask_toxicity(input_text):
    cleaned_text = normalize_slang_words(preprocess_comment(input_text))

    prediction_label = toxicity_detection_pipeline.predict([cleaned_text])[0]
    prediction_probability = toxicity_detection_pipeline.predict_proba([cleaned_text])[0][1]

    if prediction_label == 1:
        masked_output_text = advanced_toxic_word_masking(input_text)
        final_label = "Toxic ⚠️"
    else:
        masked_output_text = input_text
        final_label = "Non-Toxic ✅"

    return final_label, round(prediction_probability * 100, 2), masked_output_text


# 17. Final User Input Test Cell (Live Project Demo)
*This is the final demo cell to show how the system detects toxicity and applies masking in real time.*


In [ ]:
demo_input_comment = "fuck that idoit, నువ్వు వెధవ"

demo_label, demo_score, demo_masked_output = detect_and_mask_toxicity(demo_input_comment)

print("\n🛡️ FINAL INDUSTRY-GRADE PROJECT OUTPUT")
print("="*60)
print("Original Comment :", demo_input_comment)
print("Prediction       :", demo_label)
print("Toxicity Score   :", demo_score, "%")
print("Masked Output    :", demo_masked_output)
print("="*60)



🛡️ FINAL INDUSTRY-GRADE PROJECT OUTPUT
Original Comment : fuck that idoit, నువ్వు వెధవ
Prediction       : Toxic ⚠️
Toxicity Score   : 100.0 %
Masked Output    : **** **** idoit, నువ్వు ****


# STREAMLIT APP


In [ ]:
# Create model folder
os.makedirs("model", exist_ok=True)

# Save trained ML model
joblib.dump(toxicity_detection_pipeline, "model/toxic_model.pkl")

# Save offensive word list
joblib.dump(list(final_offensive_word_list), "model/final_bad_words.pkl")

print("✅ Model and masking data saved successfully")


✅ Model and masking data saved successfully


#INSTALLING STREAMLIT

In [ ]:
pip install streamlit

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("36fIO9OQVeboOfNIXmL3lNcGeTG_6s2oRrfvShNFLW8u9vebR")


In [ ]:
public_url = ngrok.connect(8501)
print("✅ Your Public Web App Link:", public_url)


✅ Your Public Web App Link: NgrokTunnel: "https://ryleigh-claimable-osmotically.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/content/logs.txt &